In [1]:
import requests

import pandas as pd

from io import StringIO

USER = 'secret_login'
PASSWORD = 'secret_password'

In [2]:
resp_city = requests.get('http://rachel.maykinmedia.nl/djangocase/city.csv', auth=(USER, PASSWORD))

In [3]:
resp_city.status_code

200

In [4]:
df_cities = pd.read_csv(StringIO(resp_city.text), 
                 sep=';',
                 header=None,
                 names=['id', 'city'],
)

In [5]:
df_cities

,id,city
0,AMS,Amsterdam
1,ANT,Antwerpen
2,ATH,Athene
3,BAK,Bangkok
4,BAR,Barcelona
5,BER,Berlijn


In [6]:
resp_hotel = requests.get('http://rachel.maykinmedia.nl/djangocase/hotel.csv', auth=(USER, PASSWORD))

In [7]:
resp_hotel.status_code

200

In [8]:
df_hotels = pd.read_csv(StringIO(resp_hotel.text), 
                 sep=';',
                 header=None,
                 names=['city_id', 'id', 'hotel'],
)

In [9]:
df_hotels.head(3)

,city_id,id,hotel
0,AMS,AMS01,Ibis Amsterdam Airport
1,AMS,AMS02,Novotel Amsterdam Airport
2,ANT,ANT01,Express by Holiday Inn


In [10]:
df_hotels.shape

(196, 3)

In [17]:
def df_to_database(engine, df, schema, table_name, sep='\x01', encoding='utf-8'):

    # Prepare data
    output = StringIO()
    df.to_csv(output, sep=sep, header=False, encoding=encoding, index=False)
    output.seek(0)

    # Insert data
    connection = engine.raw_connection()
    cursor = connection.cursor()
    schema_tablename = '{}.{}'.format(schema, table_name)
    cursor.copy_from(output, schema_tablename, sep=sep, null='')
    connection.commit()
    cursor.close()

In [12]:
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker

DATABASE_URI = 'postgresql+psycopg2://sandervandenoord:''@localhost/hotel_viewer'

engine = create_engine(DATABASE_URI, encoding='utf-8')

Session = sessionmaker(bind=engine)

session = Session()

/Users/sandervandenoord/miniconda2/envs/python3/lib/python3.6/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [13]:
pd.read_sql("""select * from public.locations_city""", session.bind)

,id,city


In [14]:
session.close()

In [20]:
session.execute("""TRUNCATE public.locations_city CASCADE;""")
session.commit()

In [21]:
session.execute("""TRUNCATE public.locations_hotel CASCADE;""")
session.commit()

In [22]:
df_to_database(
    engine=session.bind,
    df=df_cities,
    schema='public',
    table_name='locations_city',
)

In [23]:
df_to_database(
    engine=session.bind,
    df=df_hotels,
    schema='public',
    table_name='locations_hotel',
)